In [ ]:
import pandas as pd

import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm

import ase.io as aio
from ase.visualize import view

import numpy as np

In [ ]:
# Which reference bond? -> methanol

In [ ]:
amons_db = pd.read_pickle('/data/sahre/datasets/amons_qm9_11k/amons_database.pd')

In [ ]:
amons_db.loc[amons_db['smiles']=='CO']

In [ ]:
amons = uqm.load_obj('/data/sahre/datasets/amons_qm9_11k/unique_amons_dict')

In [ ]:
smiles = 'CF'

In [ ]:
cx = ['CC', 'CN', 'CO', 'CF']
for smiles in cx:
    with open (f'/home/sahre/work/scratch/{smiles}.xyz', 'w') as f:
        for line in amons[smiles]:
            f.write(line+'\n')

In [ ]:
cx = ['CC', 'CN', 'CO', 'CF', 'NN', 'NO', 'NF', 'OO', 'OF', 'FF']

In [ ]:
d = []
for smiles in cx:
    structure = aio.read(f'/data/sahre/projects/atomic-energies/data/bonding_trends/{smiles}.xyz')#/data/sahre/projects/atomic-energies/data/bonding_trends     /home/sahre/work/scratch
    view(structure)
    print(smiles)
    print(structure.get_distance(0,1))
    d.append(structure.get_distance(0,1))
print(np.array(d).mean())

In [ ]:
# methanol is closest tp equilibrium value

In [ ]:
# adjust all fragments -> write to new xyz-file

In [ ]:
# adjust all fragments -> write to new xyz-file
dCO = aio.read(f'/home/sahre/work/scratch/CO.xyz').get_distance(0,1)

d = []
for smiles in cx:
    structure = aio.read(f'/home/sahre/work/scratch/{smiles}.xyz')
    if structure.get_chemical_symbols()[0] == 'C':
#         structure.set_distance(1,0, dCO, indices=[0, 2, 3, 4], fix = 0)
        indices=[0, 2, 3, 4]
    elif structure.get_chemical_symbols()[0] == 'N':
#         structure.set_distance(0,1, dCO, indices=[0, 2, 3])
        indices=[0, 2, 3]
    elif structure.get_chemical_symbols()[0] == 'O':
#         structure.set_distance(0,1, dCO, indices=[0, 2])
        indices=[0, 2]
    elif structure.get_chemical_symbols()[0] == 'F':
#         structure.set_distance(0,1, dCO)
        indices=[0]
        
    structure.set_distance(1,0, dCO, indices=indices, fix = 0)
    view(structure)
    aio.write(f'/data/sahre/projects/atomic-energies/data/bonding_trends/{smiles}.xyz', structure)


In [ ]:
aio.read(f'/data/sahre/projects/atomic-energies/data/bonding_trends/CF.xyz').get_all_distances()

In [ ]:
aio.read(f'/home/sahre/work/scratch/CF.xyz').get_all_distances()

In [1]:
import pandas as pd

import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import utils_qm as uqm
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/hitp/')
import prepare_calculations as prep

import ase.io as aio
from ase.visualize import view

import numpy as np


In [5]:
smiles = 'CF'
# prepare input files
cx = ['CC', 'CN', 'CO', 'CF', 'NN', 'NO', 'NF', 'OO', 'OF']
final_mol = []
for smiles in ['FF']:#cx:

    compound_path = (f'/data/sahre/projects/atomic-energies/data/bonding_trends/{smiles}')

    # define parameters
    a1 = 0
    a2 = 1
    a3 = 2

    atoms = aio.read(f'/data/sahre/projects/atomic-energies/data/bonding_trends/{smiles}.xyz')
    atoms_ref = atoms.copy()

    pp_dir = '/data/sahre/PP_LIBRARY/'
    pp_type = '_GH_PBE'
    template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe_small_lambda.inp'
    template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe.inp'


    prep.wrapper_aligned(a1, a2, a3, atoms, atoms_ref, compound_path, pp_dir, pp_type, template_inp, template_inp_small_lambda)

In [12]:
# F2
from prepare_calculations import get_boxsize, get_gpts, align_molecule
def get_pos_F2():
    a1 = 0
    a2 = 1
    a3 = 2

    atoms = aio.read(f'/data/sahre/projects/atomic-energies/data/bonding_trends/FF_dummy.xyz')
    atoms_ref = atoms.copy()
    atom_symbols = atoms.get_chemical_symbols()
    nuc_charges = atoms.get_atomic_numbers()
    num_ve = 14#eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = get_gpts(num_ve) # get gridpoints
    num_gpts = num_gpts_higher

    # shift/rotate molecule to be in plane of grid points
    centroid_initial = np.mean(atoms_ref.get_positions(), axis=0)
    shift = np.array([boxsize,boxsize,boxsize])/2 - centroid_initial
    atoms.set_positions(atoms.get_positions() + shift)

    align_molecule(atoms, np.array([0,0,1]), a2, a3, a1)
    pos_z = atoms.get_positions()[a1, 2]
    lv = boxsize/num_gpts
    final_shift_z = np.array([0,0,int(num_gpts/2)*lv - pos_z])
    atoms.set_positions(atoms.get_positions() + final_shift_z)
    mean_x = atoms.get_positions()[:,0].mean()
    mean_y = atoms.get_positions()[:,1].mean()
    shift_xy = np.array([boxsize,boxsize,0])/2 - np.array([mean_x, mean_y, 0])
    atoms.set_positions(atoms.get_positions() + shift_xy)

    coords_final = atoms.get_positions()
    F2_pos = np.array([coords_final[0],coords_final[1]])
    return(F2_pos)


array([[6.45539155, 7.0021611 , 7.16882777],
       [7.87654265, 7.0021611 , 7.16882777]])

In [20]:
from prepare_calculations import get_lambda, write_input, write_pp_files_compound
import explore_qml_data as eqd
import os

compound_path = f'/data/sahre/projects/atomic-energies/data/bonding_trends/FF'

pp_dir = '/data/sahre/PP_LIBRARY/'
pp_type = '_GH_PBE'
template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe_small_lambda.inp'
template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe.inp'

# calculation parameters (independent of lambda value)
atoms = aio.read(f'/data/sahre/projects/atomic-energies/data/bonding_trends/FF.xyz')

atom_symbols = atoms.get_chemical_symbols()
nuc_charges = atoms.get_atomic_numbers()
num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
boxsize = get_boxsize(num_ve) # get boxsize
num_gpts_lower, num_gpts_higher = get_gpts(num_ve) # get gridpoints
num_gpts = num_gpts_higher

coords_final = get_pos_F2()

# get correct lambda value
lambda_values = np.array([0.4, 0.6, 0.8, 1.0])
for lam_val in lambda_values:
    new_lambda, scaled_ve = get_lambda(lam_val, num_ve)
    # scaled_ve is number of electrons added from pseudopotential file, the remaining electrons must be added in form of a negative charge
    charge = scaled_ve - num_ve # write input

    # create directory if necessary
    if scaled_ve < 10:
        scaled_ve_str = '0'+str(scaled_ve)
    else:
        scaled_ve_str = str(scaled_ve)
    lambda_path = os.path.join(compound_path, f've_{scaled_ve_str}/')
    os.makedirs(lambda_path, exist_ok=True)

    # generate input file
    input_path = os.path.join(lambda_path, 'run.inp')
    if new_lambda > 0.5:
        write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, template_inp, debug = False)
    else:
        write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, template_inp_small_lambda, debug = False)

    # generate pp-files
    write_pp_files_compound(atom_symbols, new_lambda, lambda_path, pp_dir, pp_type)

In [ ]:
# submit calcs, all lambda =1, then 0.8, .6,...

In [ ]:
# get atomic energies

In [ ]:
import glob

In [ ]:
from natsort import natsorted
import os


import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
import alchemy_tools2 as at
from parse_density_files import CUBE
from explore_qml_data import get_num_val_elec

def wrapper_read_data(paths):
    # load data, parameters

    densities, nuclei, gpts, hmatrix = read_cube_data(paths)
    
    # get density of ueg
    num_ve = get_num_val_elec(nuclei[:, 0])
    ueg = np.zeros(gpts[0].shape)
    ueg[:] = num_ve/gpts[0].shape[0]**3
    assert np.isclose(ueg.sum(), num_ve), 'wrong ueg'
    densities.insert(0, ueg)
    
    return(densities, gpts, hmatrix, nuclei)

def wrapper_alchpots(densities, nuclei, meshgrid, h_matrix):
    """
    integrate electron density over r for single lambda values
    """
    alchpots = []
    for d in densities:
        out = at.calculate_atomic_energies(d, nuclei, meshgrid, h_matrix, intg_method = 'sum')
        alchpots.append(out[2])
    return(np.array(alchpots))

def wrapper_atomic_energy(compound_path):
    # make path to cube files
    cubes = ['ve_06.cube', 've_08.cube', 've_11.cube', 've_14.cube']
    paths = []
    for c in cubes:
        paths.append(os.path.join(compound_path, f'cube-files/{c}'))
    
    paths = natsorted(paths)
    # read cube-files
    densities, gpts, hmatrix, nuclei = wrapper_read_data(paths)
    # calculate mu tilde
    alchpots = wrapper_alchpots(densities, nuclei, gpts, hmatrix)

    # get lambda
    lam_vals = parse_lambda(cubes)
    # integrate over lambda
    averaged_alchpots = get_mu_tilde(lam_vals, alchpots)
    
    # get Z_eff
    Z_eff = get_Zeff(nuclei[:,0])
    e_el = averaged_alchpots*Z_eff
    
    return(averaged_alchpots, e_el, nuclei, Z_eff)

def write_atomic_energies(averaged_alchpots, e_el, nuclei, Z_eff, path):
    file_path = os.path.join(path, 'atomic_energies.txt')
    header = 'Z\t Z_eff\t x\t y\t z\t mu_tilde\t e_el'
    data = np.array([nuclei[:,0], Z_eff, nuclei[:,1], nuclei[:,2], nuclei[:,3], averaged_alchpots, e_el])
    np.savetxt(file_path, data.T, delimiter='\t', header=header)
    
def wrapper_write_atomic_energies(compound_path):
    averaged_alchpots, e_el, nuclei, Z_eff = wrapper_atomic_energy(compound_path)
    write_atomic_energies(averaged_alchpots, e_el, nuclei, Z_eff, compound_path)
    
def read_cube_data(paths_cubes):
    """
    returns the data necessary to calculate the atomic energies from the cube-files
    for different lambda values
    
    paths_cubes: paths to cubes files
    densities: densities given in different cube files
    lam_vals: lambda value for cube file, obtained by parsing filename
    nuclei: charges and coordinates of the nuclei
    gpts: the gridpoints where density values are given
    """
    
    densities = []
    nuclei = None # nuclear charges and their positions
    gpts = None # gridpoints where density values are given
    h_matrix = np.zeros((3,3)) # needed for the calculation of the distance of the nuclei to the gridpoints with MIC
    
    for idx, path in enumerate(paths_cubes):
        cube = CUBE(path)
        
        densities.append(cube.data_scaled) # density
        
        fname = os.path.split(path)[1] # lambda value
        fname = fname.split('.')[0]
        total_ve = get_num_val_elec(cube.atoms[:, 0])
        # check if integrates to correct number of electrons
        if not np.isclose(cube.data_scaled.sum(), total_ve):
            print(f'{path} does not integrate to correct num_ve')
        
        if idx==len(paths_cubes)-1:
            nuclei = cube.atoms
            gpts = cube.get_grid()
            h_matrix = [cube.X*cube.NX, cube.Y*cube.NY, cube.Z*cube.NZ]
    
    return(densities, nuclei, gpts, h_matrix)

def get_Zeff(Z):
    Zeff = []
    for z in Z:
        if z < 3:
            Zeff.append(z)
        elif z > 2 and z < 11:
            Zeff.append(z-2)
        else:
            raise Exception(f"Zeff not implemented for {z}")
    return(np.array(Zeff))

def parse_lambda(cube_files):
    """
    get lamval from ve_xx.cube
    """
    lam_vals = [0.0]
    for c in cube_files:
        num_val = float(c.split('.')[0].split('_')[-1])
        lam_vals.append(num_val)
        
    lam_vals = np.array(lam_vals)
    return(lam_vals/lam_vals[-1])

def get_mu_tilde(lam_vals, alchpots):
    """
    inetgrate over tilde rho with respect to lambda
    """
    mu_tilde = []
    for i in range(alchpots.shape[1]):
        mu_tilde.append(np.trapz(alchpots[:, i], lam_vals))
    return(np.array(mu_tilde))

In [ ]:
# compound_path = '/data/sahre/projects/atomic-energies/data/bonding_trends/OO'
compound_paths = glob.glob('/data/sahre/projects/atomic-energies/data/bonding_trends/*/')

for c in compound_paths:
    wrapper_write_atomic_energies(c)